In [1]:
import pandas as pd
import numpy as np
from keras.utils import image_dataset_from_directory
import cv2

In [2]:
c=cv2.imread("/kaggle/input/tomatoleaf/tomato/train/Tomato___Bacterial_spot/01375198-62af-4c40-bddf-f3c11107200b___GCREC_Bact.Sp 5914.JPG")
c.shape

(256, 256, 3)

In [3]:
train_df="/kaggle/input/tomatoleaf/tomato/train"
train_image=image_dataset_from_directory(
train_df,
seed=123,
shuffle=True,
image_size=(256,256),
batch_size=32,
)
val_df="/kaggle/input/tomatoleaf/tomato/val"
val_image=image_dataset_from_directory(
val_df,
seed=123,
shuffle=True,
image_size=(256,256),
batch_size=32,
)

Found 10000 files belonging to 10 classes.
Found 1000 files belonging to 10 classes.


# # Deep learning


In [4]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Input,BatchNormalization,TimeDistributed,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [5]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [6]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(C)
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(C)

M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
B=BatchNormalization()(M)
D=Dropout(0.5)(B)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(C)
C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(C)

M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
B=BatchNormalization()(M)
F=Flatten()(B)
D=Dense(64,activation="relu",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(F)
D=Dense(64,activation="relu",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
D=Dense(32,activation="relu",kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(D)
outputs=Dense(10,activation='softmax')(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
model.fit(train_image,epochs=50,validation_data=val_image,callbacks=[early_stopping])

Epoch 1/50


I0000 00:00:1726461975.712939     558 service.cc:145] XLA service 0x7b99380047c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726461975.712992     558 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1726461975.712999     558 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1726461991.419311     558 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 115s 309ms/step - accuracy: 0.3116 - loss: 26.9867 - val_accuracy: 0.1000 - val_loss: 173.7316
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.5984 - loss: 7.2566 - val_accuracy: 0.1650 - val_loss: 13.1516
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.6555 - loss: 6.3955 - val_accuracy: 0.4250 - val_loss: 6.8448
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.6886 - loss: 5.7880 - val_accuracy: 0.2060 - val_loss: 9.1344
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.7509 - loss: 4.5268 - val_accuracy: 0.1410 - val_loss: 79.6054
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.7107 - loss: 5.1051 - val_accuracy: 0.5090 - val_loss: 5.1368
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.7815 - loss: 4.0137 - val_accuracy: 0.5330 - val_loss: 5.6067
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 263ms/step - accuracy: 0.7981 - loss: 3.7413

In [8]:
loss,accuracy=model.evaluate(val_image)

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.6637 - loss: 2.9500
